<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/futures/forward_curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
from io import BytesIO
from zipfile import ZipFile
import requests
import quandl
quandl.ApiConfig.api_key = "f-5zoU2G4zzHaUtkJ7BY"

url = (
    "https://data.nasdaq.com/api/v3/databases/SRF/metadata?api_key="
    + quandl.ApiConfig.api_key
)
content = requests.get(url)
zf = ZipFile(BytesIO(content.content))
file = zf.namelist()[0]
meta = pd.read_csv(zf.open(file), low_memory=False)

months = ["F", "G", "H", "J", "K", "M", "N", "Q", "U", "V", "X", "Z"]
codeToMonth = dict(zip(months, range(1, 13)))

def expiry(code):
    year = code[-4:]
    month = codeToMonth[code[-5]]
    month = str(month) if month >= 10 else "0" + str(month)
    return datetime.strptime(year + month + "21", "%Y%m%d")


meta["expiry"] = meta.code.map(expiry)
meta["contract"] = meta.code.map(lambda x: x[:-5])

In [ ]:
contract = 'ICE_T'      # example: WTI Crude Oil Futures
codes = meta.code[(meta.contract == contract) & (meta.expiry >= datetime.today())]
contracts = ["SRF/" + c for c in codes]
d = None
for c in contracts:
    try:
        x = quandl.get(c, rows=1).reset_index()
        x["Contract"] = c.split("/")[1]
        d = pd.concat((d, x))
    except:
        pass

d

,Date,Open,High,Low,Settle,Volume,Prev. Day Open Interest,Contract
0,2022-06-27,96.41,96.65,95.84,96.50,2032.0,10626.0,ICE_TF2023
0,2022-06-27,84.87,84.87,84.87,84.87,131.0,2710.0,ICE_TF2024
0,2022-06-27,77.96,77.96,77.96,77.96,0.0,1101.0,ICE_TF2025
0,2022-06-27,73.04,73.04,73.04,73.04,0.0,250.0,ICE_TF2026
0,2022-06-27,69.53,69.53,69.53,69.53,0.0,0.0,ICE_TF2027
...,...,...,...,...,...,...,...,...
0,2022-06-27,73.41,73.41,73.41,73.41,13.0,6779.0,ICE_TZ2025
0,2022-06-27,69.83,69.83,69.83,69.83,0.0,28.0,ICE_TZ2026
0,2022-06-27,67.47,67.47,67.47,67.47,0.0,0.0,ICE_TZ2027
0,2022-06-27,66.32,66.32,66.32,66.32,0.0,0.0,ICE_TZ2028
